In [1]:
#import
import pandas as pd
import numpy as np

/nfs/2017/s/ssong/.pyenv/versions/3.6.5/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/nfs/2017/s/ssong/.pyenv/versions/3.6.5/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df_trainQ = pd.read_csv('data/CsvTestxTrain/trainQuestions.csv', names=['Category', 'Question_ID', 'Title'])

In [3]:
df_trainQ.drop(df_trainQ.index[0], inplace=True)

In [4]:
df_trainQ.head()

,Category,Question_ID,Title
1,raspberrypi,0,"DS18b20 sensor disappearing and turning into ""..."
2,raspberrypi,1,Openelec/xbmc for Raspberry Pi No Addons Avail...
3,raspberrypi,2,Where does my secondary IP come from?
4,raspberrypi,3,What are the overclocking capabilities?
5,raspberrypi,4,How can I connect by raspberry using the tethe...


In [5]:
title = df_trainQ['Title'].astype(str)
title.head()

1    DS18b20 sensor disappearing and turning into "...
2    Openelec/xbmc for Raspberry Pi No Addons Avail...
3                Where does my secondary IP come from?
4              What are the overclocking capabilities?
5    How can I connect by raspberry using the tethe...
Name: Title, dtype: object

In [6]:
df_trainA = pd.read_csv('data/CsvTestxTrain/testAnswers.csv', names=['ID', 'GroupID', 'BestAnswer', 'Text'])

In [7]:
df_trainA.drop(df_trainA.index[0], inplace=True)

In [8]:
df_trainA.head()

,ID,GroupID,BestAnswer,Text
1,1,0,NaN,"<p>When you troobleshoot your hostname, you mu..."
2,2,0,NaN,"<p>You may have been able to use the hostname,..."
3,3,1,NaN,<p>The display appears to use the SPI interfac...
4,4,1,NaN,<p>The pins potentially used by the LCD module...
5,5,2,NaN,<p>You can use an arbitrary GPIO to replace th...


In [9]:
#fill NaNs with 0 to remove mixed types
df_trainA = df_trainA.fillna(0)

In [10]:
df_trainA.columns

Index(['ID', 'GroupID', 'BestAnswer', 'Text'], dtype='object')

In [11]:
df_trainA.head()

,ID,GroupID,BestAnswer,Text
1,1,0,0,"<p>When you troobleshoot your hostname, you mu..."
2,2,0,0,"<p>You may have been able to use the hostname,..."
3,3,1,0,<p>The display appears to use the SPI interfac...
4,4,1,0,<p>The pins potentially used by the LCD module...
5,5,2,0,<p>You can use an arbitrary GPIO to replace th...


In [14]:
print(df_trainA['Text'][1])

<p>When you troobleshoot your hostname, you must check some points :</p>

<ul>
<li><p><code>cat /etc/hostname</code> should show your hostname.</p></li>
<li><p>In <code>/etc/samba/smb.conf</code> file (if you use samba), <code>netbios name</code> should also point to your hostname.</p></li>
</ul>

<p>You must also check that your hostname doen'nt contain invalid characters.
Try to use one with only alphanumerics characters.</p>

<p>You may also try these commands, made for live hostname edition  : </p>

<pre><code>echo "Hostname"&gt;/etc/hostname
sudo hostname "MyHostname"
sudo /etc/init.d/networking restart
sudo systemctl daemon-reload
sudo /etc/init.d/samba restart
</code></pre>



In [15]:
#Take out text portion to clean
textA = df_trainA['Text'].astype(str)
textA.head()
html=textA.iloc[0]

In [16]:
#HTML tag remover
import re

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('',text)

newline = re.compile(r'[\n\r\t]')

def remove_newline(text):
    return newline.sub('', text)

In [17]:
#Remove tags and newlines for all text and put it in a list called new_text

new_text=[]

for i in range(0, len(textA)):
    html=textA.iloc[i]
    a=remove_tags(html)
    b=remove_newline(a)
    new_text.append(b)

In [18]:
new_text[0]

'When you troobleshoot your hostname, you must check some points :cat /etc/hostname should show your hostname.In /etc/samba/smb.conf file (if you use samba), netbios name should also point to your hostname.You must also check that your hostname doen\'nt contain invalid characters.Try to use one with only alphanumerics characters.You may also try these commands, made for live hostname edition  : echo "Hostname"&gt;/etc/hostnamesudo hostname "MyHostname"sudo /etc/init.d/networking restartsudo systemctl daemon-reloadsudo /etc/init.d/samba restart'

In [19]:
se = pd.Series(new_text)
df_trainA['CleanText'] = se.values

In [15]:
df_trainA.head()

,ID,GroupID,BestAnswer,Text,CleanText
1,1,0,0,"<p>When you troobleshoot your hostname, you mu...","When you troobleshoot your hostname, you must ..."
2,2,0,0,"<p>You may have been able to use the hostname,...","You may have been able to use the hostname, be..."
3,3,1,0,<p>The display appears to use the SPI interfac...,The display appears to use the SPI interface p...
4,4,1,0,<p>The pins potentially used by the LCD module...,The pins potentially used by the LCD module ar...
5,5,2,0,<p>You can use an arbitrary GPIO to replace th...,You can use an arbitrary GPIO to replace the a...


In [16]:
df_trainA.drop(columns=['Text']).head()

,ID,GroupID,BestAnswer,CleanText
1,1,0,0,"When you troobleshoot your hostname, you must ..."
2,2,0,0,"You may have been able to use the hostname, be..."
3,3,1,0,The display appears to use the SPI interface p...
4,4,1,0,The pins potentially used by the LCD module ar...
5,5,2,0,You can use an arbitrary GPIO to replace the a...


In [17]:
df_trainA.shape

(76830, 5)

In [ ]:
df_trainA.to_csv('data/CsvTestxTrain/CleanTestAnswers.csv', index=False)


In [127]:
#Prepare title data to analyzable data
questions = []
for i in range(0, len(title)):
    questions.append(title.iloc[i])

In [165]:
!pip install -U spaCy
!python -m spacy download en_vectors_web_lg

Requirement already up-to-date: spaCy in /nfs/2017/t/thomkim/xbrain/lib/python3.7/site-packages (2.0.12)

    Linking successful
    /nfs/2017/t/thomkim/xbrain/lib/python3.7/site-packages/en_vectors_web_lg
    -->
    /nfs/2017/t/thomkim/xbrain/lib/python3.7/site-packages/spacy/data/en_vectors_web_lg

    You can now load the model via spacy.load('en_vectors_web_lg')



In [166]:
import spacy
nlp = spacy.load('en_vectors_web_lg')

In [182]:
#Using spacy module, nlp sentences for both questions and answers 
#IOW, make sentences vectorized using pre-defined en_core_web_lg model

# answers first - new_text
nlp_text = []
for i in range(0, 1000):
    nlp_text.append(nlp(new_text[i]))

In [175]:
t = np.asarray(nlp_text)

array([Looks like there is thread here that discusses a very similar/same problem you describe. Extract from that link  by ddevejian » Thu Apr 02, 2015 2:11 am  All right, its working.    I found a couple of comments in other forums that suggested that the "Family 0 for 00-XX00000000000.YY is not registered" message is not an error, but typical behavior for the 1wire driver when nothing is attached.     Based off of that, I reattached my circuit and started to double check everything. Looking at the datasheet for the DS18B20, the input voltage is 3.0 to 5.5. Thinking that perhaps the 3v3 pin wasnt delivering sufficient voltage for whatever reason, I tried the 5v0 on pin 2. Still no love. However, when I changed the ground pin from pin 9 to pin 6, low and behold it started working. (Ive since moved back to the 3v3 on pin 1 for power and it continued to work.) For the moment Im going to assume there is a break in my ribbon cable or the T-Connector I used to connect the GPIO to the breadb

In [183]:
len(t)

1000

In [189]:
df = df_trainQ[0:1000]

In [190]:
len(df)

1000

In [191]:
df["nlp_q"] = t

/nfs/2017/t/thomkim/xbrain/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [193]:
df.head()

,Category,Question_ID,Title,nlp_q
1,raspberrypi,0,"DS18b20 sensor disappearing and turning into ""...","(Looks, like, there, is, thread, here, that, d..."
2,raspberrypi,1,Openelec/xbmc for Raspberry Pi No Addons Avail...,"(I, 've, had, this, problem, in, the, past, an..."
3,raspberrypi,2,Where does my secondary IP come from?,"(After, having, had, the, same, problem, ,, I,..."
4,raspberrypi,3,What are the overclocking capabilities?,"(Looking, at, the, datasheet, for, the, DS18B2..."
5,raspberrypi,4,How can I connect by raspberry using the tethe...,"(When, I, did, a, test, run, of, this, on, Ope..."


In [168]:
nlp_text[0]

Looks like there is thread here that discusses a very similar/same problem you describe. Extract from that link  by ddevejian » Thu Apr 02, 2015 2:11 am  All right, its working.    I found a couple of comments in other forums that suggested that the "Family 0 for 00-XX00000000000.YY is not registered" message is not an error, but typical behavior for the 1wire driver when nothing is attached.     Based off of that, I reattached my circuit and started to double check everything. Looking at the datasheet for the DS18B20, the input voltage is 3.0 to 5.5. Thinking that perhaps the 3v3 pin wasnt delivering sufficient voltage for whatever reason, I tried the 5v0 on pin 2. Still no love. However, when I changed the ground pin from pin 9 to pin 6, low and behold it started working. (Ive since moved back to the 3v3 on pin 1 for power and it continued to work.) For the moment Im going to assume there is a break in my ribbon cable or the T-Connector I used to connect the GPIO to the breadboard. D

In [196]:
# questions

nlp_questions = []

for i in range(0, 1000):
    nlp_questions.append(nlp(questions[i]))

In [197]:
p = np.asarray(nlp_questions)

In [195]:
dfa = df_trainA[0:1000]
dfa.head()

,ID,GroupID,BestAnswer,Text
1,1,0,0,"<p>Looks like there is thread <a href=""https:/..."
2,2,0,0,<p>I've had this problem in the past and it wa...
3,3,0,0,"<p>After having had the same problem, I found ..."
4,4,0,0,"<p>Looking at the datasheet for the DS18B20, t..."
5,5,1,0,<p>When I did a test run of this on OpenElec X...


In [172]:
nlp_questions[2]

Where does my secondary IP come from?

In [173]:
b.similarity(a)

0.8798156558068911